# Data Sources

In [37]:
from newsgac import database
from newsgac.data_sources.models import DataSource
from newsgac.genres import genre_labels
from pandas import DataFrame

In [14]:
DataSource._mongometa.fields_dict

{'_id': <pymodm.fields.ObjectIdField at 0x7ff20996a810>,
 'articles': <pymodm.fields.EmbeddedDocumentListField at 0x7ff20996a4d0>,
 'created': <pymodm.fields.DateTimeField at 0x7ff20996a650>,
 'description': <pymodm.fields.CharField at 0x7ff20996a410>,
 'display_title': <pymodm.fields.CharField at 0x7ff20996a3d0>,
 'file': <pymodm.fields.FileField at 0x7ff20996a450>,
 'filename': <pymodm.fields.CharField at 0x7ff20996a350>,
 'task': <pymodm.fields.EmbeddedDocumentField at 0x7ff20996a5d0>,
 'training_purpose': <pymodm.fields.BooleanField at 0x7ff20996a490>,
 'updated': <pymodm.fields.DateTimeField at 0x7ff20996a690>,
 'user': <pymodm.fields.ReferenceField at 0x7ff20996a310>}

In [18]:
DataSource.articles.related_model._mongometa.fields_dict

{'date': <pymodm.fields.DateTimeField at 0x7ff2099d9910>,
 'label': <pymodm.fields.IntegerField at 0x7ff20996a210>,
 'raw_text': <pymodm.fields.CharField at 0x7ff2099d9950>,
 'year': <pymodm.fields.IntegerField at 0x7ff2099d99d0>}

In [4]:
[ds.display_title for ds in DataSource.objects.all()]

[u'NGBS Training']

In [25]:
ds = DataSource.objects.first()
print '# articles: %s' % len(ds.articles)
print 'Created by %s' % str(ds.user.name)

# articles: 813
Created by test


In [32]:
print ds.articles[1].label
print genre_labels[ds.articles[1].label]

11
Brief


## Plot

In [44]:
from bokeh.models import (
    ColumnDataSource,
    LabelSet,
    HoverTool,
    LinearColorMapper,
    PrintfTickFormatter,
    ColorBar,
    FixedTicker, FactorRange)
from bokeh.embed import components
from bokeh.plotting import figure, show
from bokeh.layouts import gridplot

from bokeh.transform import factor_cmap
from bokeh.palettes import Category20 

from bokeh.io import output_notebook
output_notebook()

def visualize_data_source_stats(data_source):
    # https://bokeh.pydata.org/en/latest/docs/user_guide/categorical.html#nested-categories
    title = data_source.display_title
    articles = [{
        'year': str(article.year),
        'label': genre_labels[article.label]
    } for article in data_source.articles]

    df = DataFrame(articles)

    group_sizes = df.groupby(['year', 'label']).size().to_frame('size')

    counts = list(group_sizes['size'])
    x = list(group_sizes.index)  # year,label tuples

    source = ColumnDataSource(data=dict(
        x=x,
        counts=counts
    ))

    p = figure(x_range=FactorRange(*x),
               plot_height=400,
               plot_width=1200,
               title="Distribution of genres over time in " + title,
               toolbar_location='right',
               tools="save,pan,box_zoom,reset,wheel_zoom"
               )

    p.vbar(source=source,
           x='x',
           top='counts',
           width=1,
           line_color="white",
           fill_color=factor_cmap('x',
                                  palette=Category20[10],
                                  factors=sorted(list(df.label.unique())),
                                  start=1,
                                  end=2
                                  )
           )

    p.x_range.range_padding = 0.1
    p.xgrid.grid_line_color = None
    p.yaxis.axis_label = "Total number of instances"
    p.yaxis.major_label_text_font_size = '10pt'
    p.xaxis.major_label_text_font_size = '10pt'
    p.xaxis.major_label_orientation = "vertical"
    p.legend.location = "top_left"
    p.legend.orientation = "horizontal"
    return p

Loading BokehJS ...

In [45]:
show(visualize_data_source_stats(ds))

## Unicode
Using print will properly display unicode characters in the Notebook, otherwise they will be escaped

In [22]:
ds.articles[1].raw_text

u"Margaret \u2022 fi\xa3Lft \xbb 'lk dacht dat | de rSffti ! democratie \\.y langza\xa3\xa3\xc9i ? merhand genoeg gevorderd was \u2022 om kleinzielige vertoningen'als het afgeketste huwelijk tussen Margaret en Peter To'wnsend overbodig te 'maken . Het `` valt me tegen , - dat : de Prinses niet het goede voorbeeld van haar oom , de hertog van Windsor , heeft gevolgd , die er zelfs de ningskroon voor liet schie- ' ten . En dat alleen vanwege het sprookje van het zogenaamde adellijke bloed ! ' -\u25a0 : \u25a0 \u2022 LOES SCHIPPERS Arnhem . ( In hoeverre inderdaad de religieuze bezwaren tegen een huwelijk met een gescheiden man bij \u25a0 Margaret \u25a0 hebben meegespeeld staat ons -als buitenstaander niet ter beoordeling . Red . HVV . ) '"

In [23]:
print ds.articles[1].raw_text

Margaret • fi£Lft » 'lk dacht dat | de rSffti ! democratie \.y langza££Éi ? merhand genoeg gevorderd was • om kleinzielige vertoningen'als het afgeketste huwelijk tussen Margaret en Peter To'wnsend overbodig te 'maken . Het `` valt me tegen , - dat : de Prinses niet het goede voorbeeld van haar oom , de hertog van Windsor , heeft gevolgd , die er zelfs de ningskroon voor liet schie- ' ten . En dat alleen vanwege het sprookje van het zogenaamde adellijke bloed ! ' -■ : ■ • LOES SCHIPPERS Arnhem . ( In hoeverre inderdaad de religieuze bezwaren tegen een huwelijk met een gescheiden man bij ■ Margaret ■ hebben meegespeeld staat ons -als buitenstaander niet ter beoordeling . Red . HVV . ) '
